## Operation class

In [70]:
class Operation():
    
    def __init__(self, input_nodes=[]):
        
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
            
    def compute(self):
        pass

In [71]:
class add(Operation):
    
    def __init__(self, x, y):
        
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        
        self.inputs = [x_var,y_var]
        return x_var + y_var

In [72]:
class multiply(Operation):
    
    def __init__(self, x, y):
        
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        
        self.inputs = [x_var,y_var]
        return x_var * y_var

In [73]:
class matmul(Operation):
    
    def __init__(self, x, y):
        
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        
        self.inputs = [x_var,y_var]
        return x_var.dot(y_var)

In [74]:
# A placeholder is simply an empty node that needs a value to be provided to compute the output. Variables are changeable
# parameter of graph
# Graph is a global variable connecting variables and placeholders to operations

In [75]:
class Placeholder():
    """
    Simply an empty node
    """
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [76]:
class Variable():
    """
    Variables can be thought of as the weights in the neural network
    """  
    def __init__(self, initial_value=None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [77]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_deafult(self):
        
        global _default_graph # this would set this default graph as a global variable named _default_graph
        _default_graph = self

Lets assume we have the following formula:
`z = Ax + b`
Lets consider A = 10 and b = 1
Then we get `z = 10x + 1`

Note that in the above formula `x` comes across as a `placeholder`.

In [78]:
g = Graph()

In [79]:
g.set_as_deafult()

In [80]:
A = Variable(10)

In [81]:
b = Variable(1)

In [82]:
# as you can see here A and b are variables which also means that A and b are the weights that we can tweak

In [83]:
x = Placeholder()

In [84]:
y = multiply(A, x)


In [85]:
z = add(y, b)

In [86]:
# Now note here that alhtough the graph is totally ready we still have not executed anything yet. Thats because remember that 
# we still do not have the value of the placeholder in place.

We are going to execute all this inside a session class in the following way

What we are going to do is execute a post order tree traversal to make sure that we execute the nodes in the correct order

In [87]:
def traverse_postorder(operation):
    """
    PostOrder Traversal of Nodes. Basically makes sure computations are done in the correct order (Ax first, then Ax + b). 
    """
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation) # this is essentially going to call the recurse function with the operation as the input
    return nodes_postorder

In [108]:
class Session():
    
    def run(self, operation, feed_dict={}):
        
        # the operation itself is going to be the operation to compute and the feed_dict is the dictionary mapping placeholders
        # to input values. feed_dict is actually the exact terminology used in tensorflow
        
        # we are going to use the feed_dict to provide values to the placeholders
        # later on we are going to feed our neural network batches of data and we are going to use the feed dict for that
        nodes_postorder = traverse_postorder(operation)
        for node in nodes_postorder:
            if type(node) == Placeholder:
                print("node is now a placeholder")
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                print("node is now a variable")
                node.output = node.value
                
            else:
                #operation
                print("node is now an operation")
                node.inputs = []
                for input_node in node.input_nodes:
                    print("type of input_node: " + str(type(input_node)))
                    print("output of input_node: " + str(input_node.output))
                    node.inputs.append(input_node.output)
                    
#                 node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
        
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        return operation.output
                

In [109]:
sess = Session()

In [110]:
result = sess.run(operation=z, feed_dict={x:15}) # here x is a placeholder remember

node is now a variable
node is now a placeholder
node is now an operation
type of input_node: <class '__main__.Variable'>
output of input_node: 10
type of input_node: <class '__main__.Placeholder'>
output of input_node: 15
node is now a variable
node is now an operation
type of input_node: <class '__main__.multiply'>
output of input_node: 150
type of input_node: <class '__main__.Variable'>
output of input_node: 1
